In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
#como el navegador a usar es chrome, hemos descargado ya el driver de la página, guardamos la ruta en una variable
#chrome_path = 'chromedriver_linux64/chromedriver'
#driver = webdriver.Chrome(executable_path = chrome_path)

In [2]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36'}

¿Qué queremos obtener de la página?
1. Nombre del producto
2. Precio (?)
3. Detalles
4. Ingredientes

## 1 OBTENER TODAS LAS URLS DE LOS PRODUCTOS

Para ello, utilizaremos bucles que iteren sobre los elementos de la página.
* Variable global cleansers_url, es el contenedor de todas las urls.
* Los links están dentro de un tag 'a', como sólo nos interesan aquellas que tengan href, pues href=True. 
* Se ha agregado class='product-image" porque si no, el bucle cogía todas las urls (wishlist, url de la imagen etc), y solo nos interesan los productos.
* Append cada uno de los links al contenedor.

In [3]:
moisturizer_url = []

In [4]:
for i in range (1,87):
    url = f'https://www.stylevana.com/en_EU/skincare/face-care/moisturizer-cream.html?p={i}' 
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'lxml')
    moisturizer_list = soup.find_all('div', class_='product-item-info')
    for item in moisturizer_list:
        for link in item.find_all('a', class_='product-image', href=True): 
            moisturizer_url.append(link['href'])
print(len(moisturizer_url))

1026


with open("cleansers_url.txt", "w") as output:
    output.write(str(cleansers_url))

In [5]:
len(moisturizer_url)

1026

## 2 OBTENER LA INFO DE LA PÁGINA PROPIA DEL PRODUCTO
Info que necesitamos:
* Nombre del producto
* Details
* Ingredients

* Details e Ingredients se encuentran dentro de un grupo <div class='panel-group'

In [9]:
moisturizer_url[206]

'https://www.stylevana.com/en_EU/caolion-blackhead-steam-pore-pack-50g.html'

In [6]:
moisturizers = []

In [7]:
for link in moisturizer_url[0:1027]:
    r = requests.get(link, headers=headers) #para cada link en el listado, hacemos una requests
    soup = BeautifulSoup(r.content, 'lxml')
    #usamos try, except porque hay ocasiones en las que no hay info sobre los ingredientes
    try:
        name = soup.find('h1', class_='product-name-h1').text
        details = (soup.find('div', id='product-view-details').text.strip()) # details
        ingredients = soup.find_all('div', id="product-view-ingredients") #ingredients
        for item in ingredients:
            ingredients = item.find('div', class_='panel-body').text.strip()
    except:
        name = 'no info'
        details = 'no info'
        ingredients = 'no info'
        
    moisturizer_info = {
    'name': name,
    'details': details,
    'ingredients': ingredients}
    
    moisturizers.append(moisturizer_info)

In [8]:
print(len(moisturizers))

1026


## 3. PASARLO A DATAFRAME

In [9]:
moisturizer_df = pd.DataFrame(moisturizers)
moisturizer_df

,name,details,ingredients
0,iUNIK - Centella Calming Gel Cream - 60ml,"Usage: Day, Night\n\n\nProduct Type: Moisturiz...",[]
1,ROVECTIN - Clean Lotus Water Cream - 60ml,"Usage: Day, Night\n\n\nProduct Type: Moisturiz...","nelumbo nucifera flower extract, glycerin, pen..."
2,SOME BY MI - AHA-BHA-PHA 30 Days Miracle Cream,"Usage: Day, Night\n\n\nProduct Type: Moisturiz...",Centella Asiatica (Gotu Kola) Extract (70.78%)...
3,innisfree - Jeju Cherry Blossom Jelly Cream,"Usage: Day, Night\n\n\nProduct Type: Moisturiz...","WATER / AQUA / EAU, DIPROPYLENE GLYCOL, NIACIN..."
4,Etude House - Soon Jung 2x Barrier Intensive C...,Product Type: Moisturizer & Cream\n\n\nProduct...,"Water, Propanediol, Pentaerythrityl Tetraethyl..."
...,...,...,...
1021,CNP LABORATORY - Dual Balance Waterlock Moist ...,"Usage: Day, Night\n\n\nProduct Type: Moisturiz...","Water, Butylene Glycol, Betaine, Cyclopentasil..."
1022,innisfree - Cauliflower Mushroom Vital Cream -...,"Usage: Day, Night\n\n\nProduct Type: Moisturiz...",[]
1023,Dr.G - Gowoonsesang Actifirm Recovery Cream - ...,"Usage: Day, Night\n\n\nProduct Type: Moisturiz...",[]
1024,O HUI - Age Recovery Cream - 50ml,"Usage: Day, Night\n\n\nProduct Type: Moisturiz...",[]


In [11]:
moisturizer = moisturizer_df.to_csv('data/moisturizer.csv')